# Load data

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [2]:
train = pd.read_parquet("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_train_with_alphanumeric_dummy_ids.parquet")
valid = pd.read_parquet("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_valid_with_alphanumeric_dummy_ids.parquet")
dummy_users = pd.read_csv("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_dummy_users_with_alphanumeric_dummy_ids.csv", header=None).values.flatten().astype(str)
products = pd.read_csv("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_productIds.csv", header=None).values.flatten().astype(int)

In [3]:
products

array([ 8650774,  9306139,  9961521, ..., 12058614, 12058615, 11927550])

# Define a Recommender model

Embeding layers gives list of random numbers for each user and each product

In [4]:
embed1 = tf.keras.layers.Embedding(5,8)

In [5]:
dummy_users_embedding = tf.keras.layers.Embedding(len(dummy_users),6)
products_embedding =  tf.keras.layers.Embedding(len(products),6)

In [6]:
dummy_users_embedding(1)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([-0.00412284, -0.02905357, -0.02707316, -0.00170527, -0.0430899 ,
       -0.04949467], dtype=float32)>

In [7]:
class SimpleRecommender(tf.keras.Model):
    def __init__(self, dummy_users, products,length_of_embedding):
        super(SimpleRecommender, self).__init__()
        self.products = tf.constant(products, dtype=tf.int32)
        self.dummy_users = tf.constant(dummy_users, dtype=tf.string)
        self.dummy_user_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.dummy_users, range(len(dummy_users))), -1)
        self.product_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.products, range(len(products))), -1)
        
        self.user_embedding = tf.keras.layers.Embedding(len(dummy_users), length_of_embedding)
        self.product_embedding = tf.keras.layers.Embedding(len(products), length_of_embedding)
        self.dot = tf.keras.layers.Dot(axes=-1)
        
    def call(self, inputs):
        user = inputs[0]
        products = inputs[1]

        user_embedding_index = self.dummy_user_table.lookup(user)
        product_embedding_index =self.product_table.lookup(products)

        user_embedding_values = self.user_embedding(user_embedding_index)

        product_embedding_value = self.product_embedding(product_embedding_index)
        return tf.squeeze(self.dot([user_embedding_values,product_embedding_value]),1)
    
    @tf.function
    def call_item_item(self, product):
        product_x = self.product_table.lookup(product)
        pe = tf.expand_dims(self.product_embedding(product_x), 0)
        
        all_pe = tf.expand_dims(self.product_embedding.embeddings, 0)#note this only works if the layer has been built!
        scores = tf.reshape(self.dot([pe, all_pe]), [-1])
        
        top_scores, top_indices = tf.math.top_k(scores, k=100)
        top_ids = tf.gather(self.products, top_indices)
        return top_ids, top_scores

# Creating a dataset

In [8]:
dummy_user_tensor = tf.constant(train[["dummyUserId"]].values, dtype=tf.string)
product_tensor = tf.constant(train[["productId"]].values, dtype=tf.int32)

dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor))
for x, y in dataset:
    print(x)
    print(y)
    break

tf.Tensor([b'PIXcm7Ru5KmntCy0yA1K'], shape=(1,), dtype=string)
tf.Tensor([10524048], shape=(1,), dtype=int32)


In [9]:
class Mapper():
    
    def __init__(self, possible_products, num_negative_products):
        self.num_possible_products = len(possible_products)
        self.possible_products_tensor = tf.constant(possible_products, dtype=tf.int32)
        
        self.num_negative_products = num_negative_products
        self.y  = tf.one_hot(0,num_negative_products+1)
    
    def __call__(self, user, product):
        random_negatives_indexs = tf.random.uniform((self.num_negative_products,),minval =0,maxval=self.num_possible_products,dtype=tf.int32)
        negatives = tf.gather(self.possible_products_tensor, random_negatives_indexs)
        candidates =tf.concat([product,negatives],axis=0)
        return (user, candidates),self.y

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor)).map(Mapper(products,10))

for (u,c),y in dataset:
    print(u)
    print(c)
    print(y)
    break

tf.Tensor([b'PIXcm7Ru5KmntCy0yA1K'], shape=(1,), dtype=string)
tf.Tensor(
[10524048 10265059 11735604 11803153 12314020  9887144  9122663 10107872
 11216266 11674419 11419427], shape=(11,), dtype=int32)
tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(11,), dtype=float32)


In [11]:
def get_dataset(df,products, num_negative_products):
    dummy_user_tensor = tf.constant(df[['dummyUserId']].values,dtype=tf.string)
    product_tensor = tf.constant(df[['productId']].values,dtype=tf.int32)

    dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor,product_tensor))
    dataset = dataset.map(Mapper(products,num_negative_products))
    dataset = dataset.batch(1024)
    return dataset

In [12]:
for (u,c),y in get_dataset(train,products,4):
    print(u)
    print(c)
    print(y)
    break

tf.Tensor(
[[b'PIXcm7Ru5KmntCy0yA1K']
 [b'd0RILFB1hUzNSINMY4Ow']
 [b'Ebax7lyhnKRm4xeRlWW2']
 ...
 [b'xuX9n8PHfSR0AP3UZ8ar']
 [b'iNnxsPFfOa9884fMjVPJ']
 [b'aD8Mn12im8lFPzXAY41P']], shape=(1024, 1), dtype=string)
tf.Tensor(
[[10524048 11202337 10175459  9267621 11129213]
 [ 9137713 10401835 10432694 11184268  7538026]
 [ 5808602 12107606 10102321 11856416 12237853]
 ...
 [11541336  8317569 10188246 10220965 10958211]
 [ 7779232 10410505  6430186  9912805  8688906]
 [ 4941259 12227114  9526476 12402983 10677510]], shape=(1024, 5), dtype=int32)
tf.Tensor(
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]], shape=(1024, 5), dtype=float32)


# Train a model

In [ ]:
model = SimpleRecommender(dummy_users, products,15)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
,optimizer=tf.keras.optimizers.SGD(learning_rate=100.),
metrics=[tf.keras.metrics.CategoricalAccuracy()])


model.fit(get_dataset(train, products,100),validation_data=get_dataset(valid, products,100),epochs=5)


Epoch 1/5
162/162 [==============================] - 23s 134ms/step - loss: 4.6151 - categorical_accuracy: 0.0136 - val_loss: 4.6151 - val_categorical_accuracy: 0.0194
Epoch 2/5
162/162 [==============================] - 13s 83ms/step - loss: 4.6119 - categorical_accuracy: 0.0846 - val_loss: 4.6147 - val_categorical_accuracy: 0.0493
Epoch 3/5
162/162 [==============================] - 9s 54ms/step - loss: 4.5990 - categorical_accuracy: 0.1931 - val_loss: 4.6075 - val_categorical_accuracy: 0.0736
Epoch 4/5
108/162 [===================>..........] - ETA: 2s - loss: 4.5475 - categorical_accuracy: 0.2473

In [ ]:
test_product =11698965


In [ ]:
print("Recs for item {}: {}".format(test_product, model.call_item_item(tf.constant(test_product, dtype=tf.int32))))


# Save the model

In [ ]:
model_path = "models/recommender/1"
inpute_signature = tf.TensorSpec(shape=(), dtype=tf.int32)
signatures = { 'call_item_item': model.call_item_item.get_concrete_function(inpute_signature)}
tf.saved_model.save(model, model_path,signatures = signatures)

In [ ]:
PATH ='C:/Users/AbuTair/Desktop/Workspace/Data Science/Outfit-predication/code/models/recommender/1'
imported_model = tf.saved_model.load(PATH)
list(imported_model.signatures.keys())


In [ ]:
result_tensor = imported_model.signatures['call_item_item'](tf.constant([11698965]))

from IPython.core.display import HTML

def path_to_image_html(path):
    return '<img src="' + path + '" width="60" >'

result_df = pd.DataFrame(result_tensor['output_0'].numpy(),columns=['ProductUrl']).head(10)

HTML(result_df.to_html(escape=False ,formatters=dict(column_name_with_image_links=path_to_image_html)))


In [ ]:
result_df.sort_values(by='ProductUrl',ascending=False)